<a href="https://colab.research.google.com/github/IamJac/MachineLearningAlgorithms/blob/main/Titanic(0_7)01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UPLOADING FILES FROM COMPUTER MEMORY**

In [1]:
from google.colab import files
file1=files.upload()
file2=files.upload()

Saving train.csv to train.csv


Saving test.csv to test.csv


**CONVERTING THE DATA INTO DATAFRAMES**

In [21]:
import pandas as pd
df1=pd.read_csv('train.csv')
df2=pd.read_csv('test.csv')

**EXPLORATORY DATA ANALYSIS**

In [29]:
#A PEEK INTO THE TESTING DATA(Data from test.csv)
print(df2.loc[0:4])
print(df2.shape)
print(df2.columns)
print("UNIQUE ELEMENTS IN THE FOLLOWING COLUMNS")
print("Embarked",df2['Embarked'].nunique())
print("Cabin",df2['Cabin'].nunique())
print("Ticket",df2['Ticket'].nunique())
print("Names",df2['Name'].nunique())
print('Sex',df2['Sex'].nunique())
print(df2.info())
test_x=df2.drop(['Name','Ticket','Cabin','Embarked'],axis=1)#This is the testing data to generate predictions for submissions
print("A Look At Test_x")
print(test_x.head())
print(test_x.shape)
print(test_x.columns)

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
(418, 11)
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
UNIQUE ELEMENTS IN THE FOL

In [43]:
#A PEEK INTO THE TRAINING DATA(Data from train.csv)
print(df1.loc[0:4])
print(df1.shape)
print(df1.info())
print("UNIQUE ELEMENTS IN THE FOLLOWING COLUMNS")
print("Embarked",df2['Embarked'].nunique())
print("Cabin",df2['Cabin'].nunique())
print("Ticket",df2['Ticket'].nunique())
print("Names",df2['Name'].nunique())
print('Sex',df2['Sex'].nunique())
train_x=df1.drop(['Name','Ticket','Cabin','Embarked','Survived'],axis=1)
trainy=df1['Survived']
train_y=pd.DataFrame(trainy)
train_y.columns=['Survived']
print("A Look At Train_x")
print(train_x.head())
print(train_x.shape)
print("A look at train_y")
print(train_y.head())
print(train_y.shape)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
(8

**DATA PREPROCESSING**

In [245]:
numerical_columns=train_x.select_dtypes(include=['int64','float64']).columns
categorical_columns=train_x.select_dtypes(include=['object']).columns
print(" Numerical columns =",numerical_columns)
print("Categorical columns =",categorical_columns)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV
numerical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant')),
    ('scaler',RobustScaler())
])
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(transformers=[
    ('num',numerical_transformer,numerical_columns),
    ('cat',categorical_transformer,categorical_columns)
])
training_x,testing_x,val_train,val_test=train_test_split(train_x,train_y,test_size=0.20,random_state=0)
print("Training_X data",training_x.shape)
print("Training y",val_train.shape)
print("Validation_X",testing_x.shape)
print("Validation_Y",val_test.shape)
print()

#OBTAINING HYPERPARAMETERS FOR THE MODEL
Preprocessed_training_x=preprocessor.fit_transform(training_x)
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
param_grid={
    "n_estimators": randint(100, 1000),
    "learning_rate": uniform(0.001, 0.01),
    "max_depth": randint(3, 40),
    "subsample": uniform(0.1, 0.9),
    "colsample_bytree": uniform(0.1, 0.9),
    "min_child_weight": randint(1, 10),
    'reg_alpha': uniform(0.1, 0.9),
    'reg_lambda': uniform(0.1, 0.9),
    'gamma': uniform(0.1, 0.9)
}
tuning_model=XGBClassifier()
grid_search=RandomizedSearchCV(tuning_model, param_grid,n_iter=30, cv=2, scoring='roc_auc_ovr', n_jobs=-1,verbose=2,error_score='raise',random_state=0)
grid_search.fit(Preprocessed_training_x,val_train)
best=grid_search.best_params_
print("BEST HYPER-PARAMETERS TO BE USED")
best_parameters=pd.DataFrame(best,index=[0])
print(best_parameters)


 Numerical columns = Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Categorical columns = Index(['Sex'], dtype='object')
Training_X data (712, 7)
Training y (712, 1)
Validation_X (179, 7)
Validation_Y (179, 1)

Fitting 2 folds for each of 30 candidates, totalling 60 fits
BEST HYPER-PARAMETERS TO BE USED
   colsample_bytree     gamma  learning_rate  max_depth  min_child_weight  \
0          0.706294  0.974751       0.009782         16                 1   

   n_estimators  reg_alpha  reg_lambda  subsample  
0           301   0.117989     0.49754   0.981628  


**MODEL CREATION AND TRAINING**

In [246]:
model=XGBClassifier(**best)
my_pipeline=Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("model",model)
])
my_pipeline.fit(training_x,val_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unkno...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.00978193471347177, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=16,
                               max_leaves=None, min_child_weight=1, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=301, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

**ASSESSING THE MODEL**

Model performance on validation data

In [247]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
Preprocessed_testing_x=preprocessor.transform(testing_x)
predictions=my_pipeline.predict(testing_x)
preds=pd.DataFrame({'Actual':val_test.values.ravel(),'Predicted':predictions.ravel()})
print(preds.loc[0:5])
print("% Accuracy = ",(accuracy_score(val_test,predictions))*100,"%")
print("Confusion Matrix")
print(confusion_matrix(val_test,predictions))
print("Classification Report")
print(classification_report(val_test,predictions))

   Actual  Predicted
0       0          0
1       0          0
2       0          0
3       1          1
4       1          0
5       1          0
% Accuracy =  85.47486033519553 %
Confusion Matrix
[[103   7]
 [ 19  50]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       110
           1       0.88      0.72      0.79        69

    accuracy                           0.85       179
   macro avg       0.86      0.83      0.84       179
weighted avg       0.86      0.85      0.85       179



**Testing the model on the whole training data**

In [248]:
preds_on_train_set=my_pipeline.predict(train_x)
print("MODEL PERFFORMANCE ON THE TRAINING DATA SET")
print("% Accuracy = ",(accuracy_score(train_y,preds_on_train_set))*100)
print("Confusion Matrix")
print(confusion_matrix(train_y,preds_on_train_set))
print("Classification Report")
print(classification_report(train_y,preds_on_train_set))
preds2=pd.DataFrame({'Actual':train_y.values.ravel(),'Predicted':preds_on_train_set.ravel()})
print(preds2.loc[0:5])

MODEL PERFFORMANCE ON THE TRAINING DATA SET
% Accuracy =  89.78675645342312
Confusion Matrix
[[527  22]
 [ 69 273]]
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       549
           1       0.93      0.80      0.86       342

    accuracy                           0.90       891
   macro avg       0.90      0.88      0.89       891
weighted avg       0.90      0.90      0.90       891

   Actual  Predicted
0       0          0
1       1          1
2       1          1
3       1          1
4       0          0
5       0          0


**MAKING PREDICTIONS USING THE TEST DATA**

In [249]:
preds_on_test_set=my_pipeline.predict(test_x)
print(preds_on_test_set.shape)
print(preds_on_test_set)

(418,)
[0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


**SAVING PREDICTIONS IN CSV FORMAT**

In [252]:
file_submit=pd.DataFrame({"PassengerId":df2.PassengerId,"Survived":preds_on_test_set})
file_submit.to_csv("file_submission2.csv",index=False)
print("Conversion Successful")

Conversion Successful


**SAVING CSV FILE IN COMPUTER MEMORY**

In [253]:
files.download("file_submission2.csv")
print("SUCCESSFUL")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESSFUL
